### run this notebook for each patient following Plastimatch DIR and calculation of the Jacobian determinant

In [1]:
import os
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import SimpleITK as sitk
import skimage
from os.path import expanduser
import glob
from matplotlib.ticker import MaxNLocator
from scipy.stats import spearmanr
import seg_metrics.seg_metrics as sg
import pickle
from skimage import exposure
from scipy import ndimage
import nibabel as nib
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from matplotlib import colors
from matplotlib import cm

In [2]:
print("Python version:", sys.version)
print("Numpy version:", np.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("SITK version:", sitk.__version__)
print("Scikit-image version:", skimage.__version__)

Python version: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
Numpy version: 1.26.2
Matplotlib version: 3.8.2
SITK version: 2.3.1
Scikit-image version: 0.22.0


In [3]:
patient_id = 14

In [4]:
PREPROCESSED_BASE_DIR = '/mnt/data/datasets/RNSH_HFlung/pre-processed-plastimatch'
PREPROCESSED_MHA_DIR = '{}/mha/Patient{:02d}'.format(PREPROCESSED_BASE_DIR, patient_id)
PREPROCESSED_STACK_DIR = '{}/stack'.format(PREPROCESSED_BASE_DIR)

#### load the Jacobian determinant and calculate the CTVI-Jac

In [5]:
# load images from the Plastimatch (pm) processing
pm_exhale_im = sitk.ReadImage('{}/phase-exhale-nomask.mha'.format(PREPROCESSED_MHA_DIR))
pm_inhale_im = sitk.ReadImage('{}/phase-inhale-nomask.mha'.format(PREPROCESSED_MHA_DIR))
pm_mask_im = sitk.ReadImage('{}/mask-union.mha'.format(PREPROCESSED_MHA_DIR))

pm_exhale_arr = sitk.GetArrayFromImage(pm_exhale_im)
pm_inhale_arr = sitk.GetArrayFromImage(pm_inhale_im)
pm_mask_arr = sitk.GetArrayFromImage(pm_mask_im).astype(bool)

jacobian_det_im = sitk.ReadImage('{}/vf_jac.mha'.format(PREPROCESSED_MHA_DIR))
jacobian_det_arr = sitk.GetArrayFromImage(jacobian_det_im)

/home/daryl/miniconda3/envs/py310/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [6]:
jacobian_det_arr.shape

(167, 512, 512)

In [7]:
prop = np.sum(np.array(jacobian_det_arr[pm_mask_arr]) < 0) / len(jacobian_det_arr[pm_mask_arr].ravel())
print('proportion of voxels with negative values in Jacobian determinant = {}'.format(round(prop, 2)))

proportion of voxels with negative values in Jacobian determinant = 0.0


In [8]:
ctvi_jac_arr = jacobian_det_arr-1.0
ctvi_jac_arr[~pm_mask_arr] = np.min(ctvi_jac_arr)
ctvi_jac_im = sitk.GetImageFromArray(ctvi_jac_arr)

# set the spacing and origin in the CTVI-Jac image
ctvi_jac_im.SetSpacing(jacobian_det_im.GetSpacing())
ctvi_jac_im.SetOrigin(jacobian_det_im.GetOrigin())

In [9]:
ctvi_jac_im.GetSize(), ctvi_jac_im.GetSpacing(), ctvi_jac_im.GetOrigin()

((512, 512, 167),
 (0.9765625, 0.9765625, 1.7999999523162842),
 (-249.51171875, -451.51171875, -527.0))

#### load the deformably registered inhalation image and calculate the CTVI-HU

In [10]:
# load images from the Plastimatch (pm) processing
pm_deformed_inhale_im = sitk.ReadImage('{}/deformed.mha'.format(PREPROCESSED_MHA_DIR))
pm_deformed_inhale_arr = sitk.GetArrayFromImage(pm_deformed_inhale_im)

In [11]:
# set all voxels to zero if there are not greater than zero
pm_exhale_arr = np.where((pm_exhale_arr>0), 0 , pm_exhale_arr)
pm_deformed_inhale_arr = np.where((pm_deformed_inhale_arr>0), 0 , pm_deformed_inhale_arr)

# apply the mask to the images
pm_exhale_arr = np.where((pm_mask_arr==True), pm_exhale_arr, 0)
pm_deformed_inhale_arr = np.where((pm_mask_arr==True), pm_deformed_inhale_arr, 0)

In [12]:
# rescale for air and water
airHU = -1000. + 1
waterHU = 0.

pm_exhale_arr = exposure.rescale_intensity(pm_exhale_arr, in_range='image', out_range=(airHU, waterHU))
pm_deformed_inhale_arr = exposure.rescale_intensity(pm_deformed_inhale_arr, in_range='image', out_range=(airHU, waterHU))

In [13]:
tissue_density_scaling_factor_arr = (pm_exhale_arr + 1000) / 1000
ctvi_hu_arr = (pm_exhale_arr - pm_deformed_inhale_arr) / (pm_deformed_inhale_arr + 1000) * tissue_density_scaling_factor_arr
ctvi_hu_im = sitk.GetImageFromArray(ctvi_hu_arr)

# set the spacing and origin in the CTVI-Jac image
ctvi_hu_im.SetSpacing(pm_exhale_im.GetSpacing())
ctvi_hu_im.SetOrigin(pm_exhale_im.GetOrigin())

In [14]:
ctvi_hu_im.GetSize(), ctvi_hu_im.GetSpacing(), ctvi_hu_im.GetOrigin()

((512, 512, 167),
 (0.9765625, 0.9765625, 1.7999999999999998),
 (-249.51171875, -451.51171875, -527.0))

#### resample, crop, & pad the CTVI-Jac so we can make a comparison with model predictions

In [15]:
# Resampling the images into (1,1,1)mm voxel size. Can use nearest neighbour for interpolation (default is B-spline). 
def resample_image(itk_image, out_spacing=(1.0, 1.0, 1.0), is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

In [16]:
ctvi_jac_im = resample_image(ctvi_jac_im)
ctvi_hu_im = resample_image(ctvi_hu_im)

In [17]:
# convert back into numpy array after resampling
ctvi_jac_arr = sitk.GetArrayFromImage(ctvi_jac_im)
ctvi_hu_arr = sitk.GetArrayFromImage(ctvi_hu_im)

In [18]:
ctvi_jac_arr.shape, ctvi_hu_arr.shape

((301, 500, 500), (301, 500, 500))

In [19]:
TARGET_SIZE_X = 320
TARGET_SIZE_Y = 320
TARGET_SIZE_Z = 300

In [20]:
# pad CTVI-Jac z axis if needed
if ctvi_jac_arr.shape[0] < TARGET_SIZE_Z:
    pad_slices = TARGET_SIZE_Z - ctvi_jac_arr.shape[0]
    ctvi_jac_arr = np.pad(ctvi_jac_arr, pad_width=((0,pad_slices),(0,0),(0,0)), mode='constant', constant_values=np.min(ctvi_jac_arr))  # pad at the back

# crop x and y axes of CT
crop_x_lower = int(np.round((ctvi_jac_arr.shape[2]-TARGET_SIZE_X)/2)); crop_x_upper = crop_x_lower+TARGET_SIZE_X
crop_y_lower = int(np.round((ctvi_jac_arr.shape[1]-TARGET_SIZE_Y)/2)); crop_y_upper = crop_y_lower+TARGET_SIZE_Y
crop_z_lower = int(np.round((ctvi_jac_arr.shape[0]-TARGET_SIZE_Z)/2)); crop_z_upper = crop_z_lower+TARGET_SIZE_Z

ctvi_jac_arr = ctvi_jac_arr[crop_z_lower:crop_z_upper, crop_y_lower:crop_y_upper, crop_x_lower:crop_x_upper]

ctvi_jac_arr.shape

(300, 320, 320)

In [21]:
# pad CTVI-HU z axis if needed
if ctvi_hu_arr.shape[0] < TARGET_SIZE_Z:
    pad_slices = TARGET_SIZE_Z - ctvi_hu_arr.shape[0]
    ctvi_hu_arr = np.pad(ctvi_hu_arr, pad_width=((0,pad_slices),(0,0),(0,0)), mode='constant', constant_values=np.min(ctvi_hu_arr))  # pad at the back

# crop x and y axes of CT
crop_x_lower = int(np.round((ctvi_hu_arr.shape[2]-TARGET_SIZE_X)/2)); crop_x_upper = crop_x_lower+TARGET_SIZE_X
crop_y_lower = int(np.round((ctvi_hu_arr.shape[1]-TARGET_SIZE_Y)/2)); crop_y_upper = crop_y_lower+TARGET_SIZE_Y
crop_z_lower = int(np.round((ctvi_hu_arr.shape[0]-TARGET_SIZE_Z)/2)); crop_z_upper = crop_z_lower+TARGET_SIZE_Z

ctvi_hu_arr = ctvi_hu_arr[crop_z_lower:crop_z_upper, crop_y_lower:crop_y_upper, crop_x_lower:crop_x_upper]

ctvi_hu_arr.shape

(300, 320, 320)

#### normalise the CTVIs

In [22]:
ctvi_jac_arr = exposure.rescale_intensity(ctvi_jac_arr, in_range='image', out_range=(1e-3,1.))
ctvi_hu_arr = exposure.rescale_intensity(ctvi_hu_arr, in_range='image', out_range=(1e-3,1.))

#### apply a median filter

In [ ]:
ctvi_jac_arr = ndimage.median_filter(ctvi_jac_arr, size=7)
ctvi_hu_arr = ndimage.median_filter(ctvi_hu_arr, size=7)

#### load the PET ground truth (unquantised and quantised)

In [ ]:
output_name = '{}/Patient{:02d}.npy'.format(PREPROCESSED_STACK_DIR, patient_id)
pt_arr = np.load(output_name)
print('loaded stack from {}'.format(output_name))

exp_arr = pt_arr[0]
insp_arr = pt_arr[1]
pet_arr = pt_arr[2]
pet_labels_arr = pt_arr[3]
mask_arr = pt_arr[4].astype(bool)

In [ ]:
pet_labels_arr.shape, mask_arr.shape

### compare unquantised CTVI-Jac with PET ground truth

In [ ]:
DISPLAY_IDX = 150

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(15,7))

exp_slice = exp_arr[DISPLAY_IDX]
ax1.imshow(exp_slice, cmap="gray", norm=colors.PowerNorm(0.5, vmin=np.percentile(exp_slice,1)))
ax1.set_title('exhale')
ax1.set_axis_off()

insp_slice = insp_arr[DISPLAY_IDX]
ax2.imshow(insp_slice, cmap="gray", norm=colors.PowerNorm(0.5, vmin=np.percentile(insp_slice,1)))
ax2.set_title('inhale')
ax2.set_axis_off()

pet_masked_slice = np.copy(np.ma.array(pet_arr[DISPLAY_IDX], mask=~mask_arr[DISPLAY_IDX]))
ax3.imshow(pet_masked_slice, cmap="hot", norm=colors.PowerNorm(0.8))
ax3.set_title('PET')
ax3.set_axis_off()

ctvi_jac_masked_slice = np.copy(np.ma.array(ctvi_jac_arr[DISPLAY_IDX], mask=~mask_arr[DISPLAY_IDX]))
ax4.imshow(ctvi_jac_masked_slice, cmap="hot", norm=colors.PowerNorm(1.8))
ax4.set_title(r'CTVI$_\text{Jac}$')
ax4.set_axis_off()

ctvi_hu_masked_slice = np.copy(np.ma.array(ctvi_hu_arr[DISPLAY_IDX], mask=~mask_arr[DISPLAY_IDX]))
ax5.imshow(ctvi_hu_masked_slice, cmap="hot", norm=colors.PowerNorm(1.2, vmin=np.percentile(ctvi_hu_masked_slice,1), vmax=np.percentile(ctvi_hu_masked_slice,98), clip=True))
ax5.set_title(r'CTVI$_\text{HU}$')
ax5.set_axis_off()

# fig.suptitle('Patient ID {}'.format(patient_id), x=0.1, y=0.75, fontsize=12, ha='left')

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.92, 0.33, 0.015, 0.34])
fig.colorbar(cm.ScalarMappable(norm=colors.PowerNorm(1.2, vmin=0, vmax=1), cmap='hot'), cax=cbar_ax, orientation='vertical', format='%0.1f')

fig.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.01, 
                    hspace=0.1)

fig_filename = '{}/figures/ct-pet-ctvi-dir-unquantised-P{:02d}.svg'.format(expanduser('~'), patient_id)
plt.savefig(fig_filename, format='svg', dpi=600, bbox_inches='tight')
print('saved figure to {}'.format(fig_filename))

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(15,4))
ax1.imshow(ctvi_jac_arr[DISPLAY_IDX], cmap="gist_gray", alpha=0.5)
ax1.imshow(pet_arr[DISPLAY_IDX], cmap="gist_yarg", alpha=0.5)
ax1.set_title('PET and CTVI-Jac superimposed')
ax1.set_axis_off()

fig.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.01, 
                    hspace=0.1)

In [ ]:
sp_jac = spearmanr(pet_arr[mask_arr], ctvi_jac_arr[mask_arr])
sp_hu = spearmanr(pet_arr[mask_arr], ctvi_hu_arr[mask_arr])
print('Spearman: {}, p-value {} (Jac), {} p-value {} (HU)'.format(np.round(sp_jac[0],2), np.round(sp_jac[1],2), np.round(sp_hu[0],2), np.round(sp_hu[1],2) ))
patient_eval_d = {'spearman_correlation_jac':sp_jac[0], 'pval_jac':sp_jac[1], 'spearman_correlation_hu':sp_hu[0], 'pval_hu':sp_hu[1]}

In [ ]:
PREPROCESSED_BASE_DIR = '/mnt/data/datasets/RNSH_HFlung/pre-processed-plastimatch'
EVALUATION_DIR = '{}/evaluation'.format(PREPROCESSED_BASE_DIR)
PATIENT_EVALUATION_DIR = '{}/Patient{:02d}'.format(EVALUATION_DIR, patient_id)

In [ ]:
if not os.path.exists(EVALUATION_DIR):
    os.makedirs(EVALUATION_DIR)

if not os.path.exists(PATIENT_EVALUATION_DIR):
    os.makedirs(PATIENT_EVALUATION_DIR)

In [ ]:
DATASET_EVALUATION_FILE = '{}/evaluation-unquantised.pkl'.format(EVALUATION_DIR)

if os.path.isfile(DATASET_EVALUATION_FILE):
    with open(DATASET_EVALUATION_FILE, 'rb') as f:
        dataset_d = pickle.load(f)
else:
    dataset_d = {}

dataset_d[patient_id] = patient_eval_d

with open(DATASET_EVALUATION_FILE, 'wb') as f:
        pickle.dump(dataset_d, f)

### quantise the CTVI and save for the next step

#### quantise the CTVI-Jac

In [ ]:
percentile_high_jac = np.percentile(ctvi_jac_arr[mask_arr],70)
percentile_medium_jac = np.percentile(ctvi_jac_arr[mask_arr],30)

In [ ]:
percentile_high_jac,percentile_medium_jac

In [ ]:
# we label voxels above the 70th percentile as 'high', between the 30th and 70th as 'medium', below the 30th as 'low', and outside the 
# mask as 'background'
ctvi_jac_label_high_arr = np.where((mask_arr==True) & (ctvi_jac_arr>percentile_high_jac), 1 , 0)
ctvi_jac_label_medium_arr = np.where((mask_arr==True) & (ctvi_jac_arr<=percentile_high_jac) & (ctvi_jac_arr>=percentile_medium_jac), 2 , 0)
ctvi_jac_label_low_arr = np.where((mask_arr==True) & (ctvi_jac_arr<percentile_medium_jac), 3 , 0)

ctvi_jac_label_arr = ctvi_jac_label_high_arr + ctvi_jac_label_medium_arr + ctvi_jac_label_low_arr

#### quantise the CTVI-HU

In [ ]:
percentile_high_hu = np.percentile(ctvi_hu_arr[mask_arr],70)
percentile_medium_hu = np.percentile(ctvi_hu_arr[mask_arr],30)

In [ ]:
percentile_high_hu,percentile_medium_hu

In [ ]:
# we label voxels above the 70th percentile as 'high', between the 30th and 70th as 'medium', below the 30th as 'low', and outside the 
# mask as 'background'
ctvi_hu_label_high_arr = np.where((mask_arr==True) & (ctvi_hu_arr>percentile_high_hu), 1 , 0)
ctvi_hu_label_medium_arr = np.where((mask_arr==True) & (ctvi_hu_arr<=percentile_high_hu) & (ctvi_hu_arr>=percentile_medium_hu), 2 , 0)
ctvi_hu_label_low_arr = np.where((mask_arr==True) & (ctvi_hu_arr<percentile_medium_hu), 3 , 0)

ctvi_hu_label_arr = ctvi_hu_label_high_arr + ctvi_hu_label_medium_arr + ctvi_hu_label_low_arr

#### visualise the images

In [ ]:
labels_d = {0:'background', 1:'high', 2:'medium', 3:'low'}

# set up the colour map
colours_d = {
    0: 'black',
    1: 'lime',
    2: 'lightseagreen',
    3: 'slategray'
}

cm = ListedColormap(colours_d.values())

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(15,7))
ax1.imshow(exp_arr[DISPLAY_IDX], cmap="gray")
ax1.set_title('exhale')
ax1.set_axis_off()

ax2.imshow(insp_arr[DISPLAY_IDX], cmap="gray")
ax2.set_title('inhale')
ax2.set_axis_off()

ax3.imshow(pet_labels_arr[DISPLAY_IDX], cmap=cm, interpolation='none')
ax3.set_title('ground truth')
ax3.set_axis_off()

ax4.imshow(ctvi_jac_label_arr[DISPLAY_IDX], cmap=cm, interpolation='none')
ax4.set_title(r'CTVI$_\text{Jac}$')
ax4.set_axis_off()

ax5.imshow(ctvi_hu_label_arr[DISPLAY_IDX], cmap=cm, interpolation='none')
ax5.set_title(r'CTVI$_\text{HU}$')
ax5.set_axis_off()

# create a patch (proxy artist) for every color 
patches = [ mpatches.Patch(color=colours_d[i], label="{l}".format(l=labels_d[i]) ) for i in [1,2,3] ]
legend = plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )

fig.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.01, 
                    hspace=0.1)

fig_filename = '{}/figures/ct-pet-ctvi-dir-quantised-P{:02d}.svg'.format(expanduser('~'), patient_id)
plt.savefig(fig_filename, format='svg', dpi=600, bbox_inches='tight')
print('saved figure to {}'.format(fig_filename))

In [ ]:
params = {}
params['font.size'] = 18
params['figure.dpi'] = 150
params['figure.figsize'] = 1,1.5

plt.rcParams.update(params)

In [ ]:
fig, ax = plt.subplots()

# create a patch (proxy artist) for every color
colours_d[4] = '#87a96b'
labels_d[4] = 'high'
patches = []
x = y = 0
patches.append(mpatches.Patch(color='black', label=r'$\bf{Galligas\ PET}$', visible=False))
for i in [1,2,3]:
    patches.append(mpatches.Patch(color=colours_d[i], label="{l}".format(l=labels_d[i])))
patches.append(mpatches.Patch(color='black', label="", visible=False))
patches.append(mpatches.Patch(color='black', label=r'$\bf{CTVI_{nnU'+u'\u2010'+'Net}}$', visible=False))
for i in [4]:
    patches.append(mpatches.Patch(color=colours_d[i], label="{l}".format(l=labels_d[i])))
legend = plt.legend(handles=patches, facecolor='black', labelcolor='w', framealpha=1)
# frame = legend.get_frame()
# frame.set_facecolor('black')
# frame.set_edgecolor('black')
plt.gca().set_axis_off()

expand=[0,0,0,0]
fig  = legend.figure
fig.canvas.draw()
bbox  = legend.get_window_extent()
bbox = bbox.from_extents(*(bbox.extents + np.array(expand)))
bbox = bbox.transformed(fig.dpi_scale_trans.inverted())

fig_filename = '{}/figures/ctvi-predictions-legend.svg'.format(expanduser('~'))
plt.savefig(fig_filename, format='svg', dpi=600, bbox_inches=bbox)
print('saved figure to {}'.format(fig_filename))

#### write out the quantised CTVI for comparison

In [ ]:
sitk.WriteImage(sitk.GetImageFromArray(ctvi_jac_label_arr), '{}/ctvi-jac-label.mha'.format(PREPROCESSED_MHA_DIR))
sitk.WriteImage(sitk.GetImageFromArray(ctvi_hu_label_arr), '{}/ctvi-hu-label.mha'.format(PREPROCESSED_MHA_DIR))